In [3]:
import pandas as pd
data = pd.read_csv('math_questions_fulldata.csv')

In [4]:
import nltk
import bs4 as bs
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [5]:
data_cut = data.iloc[0:1000]

In [6]:
data.head()

,correct,options,question,rationale,spatial representations,counting and operation,logical problem solving
0,A,"['A)32400', 'B)6000', 'C)600', 'D)60000', 'E)10']",Three birds are flying at a fast rate of 900 k...,To calculate the equivalent of miles in a kilo...,0.0,1.0,0.0
1,D,"['A)100 m', 'B)150 m', 'C)200 m', 'D)250 m', '...",A ship is leaving a port. It takes 240 seconds...,Let the length of the ship be x metres and its...,1.0,0.0,0.0
2,C,"['A)6', 'B)18', 'C)24', 'D)36', 'E)48']",A rectangular piece of cloth 2 feet wide was c...,"The question says, length of shorter piece is ...",1.0,0.0,0.0
3,B,"['A)(1,1)', 'B)(0,3)', 'C)(2,0)', 'D)(3,6)', '...","In the xy-coordinate plane, which of the follo...",For a point to satisfy the given equation for ...,1.0,0.0,0.0
4,C,"['A)7', 'B)9', 'C)13', 'D)27', 'E)45']",A travel company wants to charter a plane to t...,"Additional passengers (i.e., an integer, let's...",0.0,0.0,1.0


In [7]:
n,c = data_cut.shape

In [8]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

In [9]:
data_cut.shape

(1000, 7)

In [10]:
data_cut = data_cut.dropna()

In [11]:
data_cut.shape

(995, 7)

In [12]:
data_cut = data_cut.reset_index(drop=True)

In [13]:
n,c = data_cut.shape

In [14]:
import numpy as np
from nltk.corpus import stopwords
useless = ['x', 'xx', 'xxx']
eng_stopwords = stopwords.words('english') + useless
bag = []

for i in range(n):
    #
    #tokenize of question
    #
    question = bs.BeautifulSoup(data_cut['question'][i],features='lxml').text
    question = question.replace('.','. ')
    question = re.sub(r'\{.*\}', '', question)
    question = re.sub('[^a-zA-Z ]' ,'',question)
    question = question.lower()
    question_words = question.split()
    token_tag = pos_tag(question_words)
    wnl = WordNetLemmatizer()
    wnl_stems = []
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)
    question_wo_stopwords = [w for w in question_words if not w in stopwords.words("english")]
    
    #
    #tokenize of rationale
    #
    rationale = bs.BeautifulSoup(data_cut['rationale'][i],features='lxml').text
    rationale = rationale.replace('.','. ')
    rationale = re.sub(r'\{.*\}', '', rationale)
    rationale = re.sub('[^a-zA-Z ]' ,'',rationale)
    rationale = rationale.lower()
    rationale_words = rationale.split()
    token_tag = pos_tag(rationale_words)
    wnl = WordNetLemmatizer()
    wnl_stems = []
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)
    for w in rationale_words:
        if not w in eng_stopwords:
            question_wo_stopwords.append(w)
    bag.append(question_wo_stopwords)

In [15]:
data_cut['bag'] = bag

In [16]:
data_cut.head()

,correct,options,question,rationale,spatial representations,counting and operation,logical problem solving,bag
0,A,"['A)32400', 'B)6000', 'C)600', 'D)60000', 'E)10']",Three birds are flying at a fast rate of 900 k...,To calculate the equivalent of miles in a kilo...,0.0,1.0,0.0,"[three, birds, flying, fast, rate, kilometers,..."
1,D,"['A)100 m', 'B)150 m', 'C)200 m', 'D)250 m', '...",A ship is leaving a port. It takes 240 seconds...,Let the length of the ship be x metres and its...,1.0,0.0,0.0,"[ship, leaving, port, takes, seconds, passes, ..."
2,C,"['A)6', 'B)18', 'C)24', 'D)36', 'E)48']",A rectangular piece of cloth 2 feet wide was c...,"The question says, length of shorter piece is ...",1.0,0.0,0.0,"[rectangular, piece, cloth, feet, wide, cut, l..."
3,B,"['A)(1,1)', 'B)(0,3)', 'C)(2,0)', 'D)(3,6)', '...","In the xy-coordinate plane, which of the follo...",For a point to satisfy the given equation for ...,1.0,0.0,0.0,"[xycoordinate, plane, following, points, must,..."
4,C,"['A)7', 'B)9', 'C)13', 'D)27', 'E)45']",A travel company wants to charter a plane to t...,"Additional passengers (i.e., an integer, let's...",0.0,0.0,1.0,"[travel, company, wants, charter, plane, baham..."


In [17]:
data_cut['bag'][22]

['bills',
 'shop',
 'sells',
 'candy',
 'bars',
 'full',
 'case',
 'full',
 'case',
 'added',
 'candy',
 'bars',
 'already',
 'case',
 'fill',
 'many',
 'candy',
 'bars',
 'full',
 'case',
 'answer',
 'e']

In [18]:
clean_wnl = []
wnl = WordNetLemmatizer()
for i in range(0,n):
    if( (i+1)%500 == 0 ):
        # print progress
        print("Done with %d reviews" %(i+1)) 
    
    wnl_stems = []
    insight = data_cut['bag'][i]
    token_tag = pos_tag(insight)
    for pair in token_tag:
        res = wnl.lemmatize(pair[0],pos=get_wordnet_pos(pair[1]))
        wnl_stems.append(res)

    clean_wnl.append(' '.join(wnl_stems))
    
data_cut['clean_insight'] = clean_wnl

Done with 500 reviews


In [19]:
data_cut.head()

,correct,options,question,rationale,spatial representations,counting and operation,logical problem solving,bag,clean_insight
0,A,"['A)32400', 'B)6000', 'C)600', 'D)60000', 'E)10']",Three birds are flying at a fast rate of 900 k...,To calculate the equivalent of miles in a kilo...,0.0,1.0,0.0,"[three, birds, flying, fast, rate, kilometers,...",three bird fly fast rate kilometer per hour sp...
1,D,"['A)100 m', 'B)150 m', 'C)200 m', 'D)250 m', '...",A ship is leaving a port. It takes 240 seconds...,Let the length of the ship be x metres and its...,1.0,0.0,0.0,"[ship, leaving, port, takes, seconds, passes, ...",ship leave port take second pass channel get p...
2,C,"['A)6', 'B)18', 'C)24', 'D)36', 'E)48']",A rectangular piece of cloth 2 feet wide was c...,"The question says, length of shorter piece is ...",1.0,0.0,0.0,"[rectangular, piece, cloth, feet, wide, cut, l...",rectangular piece cloth foot wide cut lengthwi...
3,B,"['A)(1,1)', 'B)(0,3)', 'C)(2,0)', 'D)(3,6)', '...","In the xy-coordinate plane, which of the follo...",For a point to satisfy the given equation for ...,1.0,0.0,0.0,"[xycoordinate, plane, following, points, must,...",xycoordinate plane follow point must lie line ...
4,C,"['A)7', 'B)9', 'C)13', 'D)27', 'E)45']",A travel company wants to charter a plane to t...,"Additional passengers (i.e., an integer, let's...",0.0,0.0,1.0,"[travel, company, wants, charter, plane, baham...",travel company want charter plane bahamas char...


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
question = data_cut['clean_insight']
vect = CountVectorizer()
vect.fit(question)
bag = vect.transform(question)
vect_df = pd.DataFrame(bag.toarray(), columns=vect.get_feature_names())
vect_df.head()

,aa,aaanswer,aanswer,aanswere,aavgavg,aavgthe,ab,abbbba,abbc,abbcacanswer,...,zh,zinc,znumber,zoology,zthe,zyxaverage,zyxzanswer,zzyyz,zzzz,zzzzzzxyanswer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
n, c = data_cut.shape

In [22]:
data_cut['logical problem solving'].sum()

42.0

In [23]:
data_cut['category'] = [0 for i in range(n)]
for i in range(n):
    if data_cut['counting and operation'][i] == 1 :
        data_cut.loc[i, 'category'] = 1
    if data_cut['logical problem solving'][i] == 1 :
        data_cut.loc[i, 'category'] = 2
    if data_cut['spatial representations'][i] == 1 :
        data_cut.loc[i, 'category'] = 3        

In [24]:
data_cut.head()

,correct,options,question,rationale,spatial representations,counting and operation,logical problem solving,bag,clean_insight,category
0,A,"['A)32400', 'B)6000', 'C)600', 'D)60000', 'E)10']",Three birds are flying at a fast rate of 900 k...,To calculate the equivalent of miles in a kilo...,0.0,1.0,0.0,"[three, birds, flying, fast, rate, kilometers,...",three bird fly fast rate kilometer per hour sp...,1
1,D,"['A)100 m', 'B)150 m', 'C)200 m', 'D)250 m', '...",A ship is leaving a port. It takes 240 seconds...,Let the length of the ship be x metres and its...,1.0,0.0,0.0,"[ship, leaving, port, takes, seconds, passes, ...",ship leave port take second pass channel get p...,3
2,C,"['A)6', 'B)18', 'C)24', 'D)36', 'E)48']",A rectangular piece of cloth 2 feet wide was c...,"The question says, length of shorter piece is ...",1.0,0.0,0.0,"[rectangular, piece, cloth, feet, wide, cut, l...",rectangular piece cloth foot wide cut lengthwi...,3
3,B,"['A)(1,1)', 'B)(0,3)', 'C)(2,0)', 'D)(3,6)', '...","In the xy-coordinate plane, which of the follo...",For a point to satisfy the given equation for ...,1.0,0.0,0.0,"[xycoordinate, plane, following, points, must,...",xycoordinate plane follow point must lie line ...,3
4,C,"['A)7', 'B)9', 'C)13', 'D)27', 'E)45']",A travel company wants to charter a plane to t...,"Additional passengers (i.e., an integer, let's...",0.0,0.0,1.0,"[travel, company, wants, charter, plane, baham...",travel company want charter plane bahamas char...,2


## 2. Fitting a support vector machine to the data

In support vector machines, we are given a set of examples $(x_1, y_1), \ldots, (x_n, y_n)$ and we want to find a weight vector $w \in \mathbb{R}^d$ that solves the following optimization problem:

$$ \min_{w \in \mathbb{R}^d} \| w \|^2 + C \sum_{i=1}^n \xi_i $$
$$ \text{subject to } y_i \langle w, x_i \rangle \geq 1 - \xi_i \text{ for all } i=1,\ldots, n$$

`scikit-learn` provides an SVM solver that we will use. The following routine takes as input the constant `C` (from the above optimization problem) and returns the training and test error of the resulting SVM model. It is invoked as follows:

* `training_error = fit_classifier(C)`

The default value for parameter `C` is 1.0.

In [25]:
from sklearn import svm
train = vect_df[0:105]
train_labels = data_cut['category'][0:105]

def fit_classifier(C_value=1.0):
    clf = svm.LinearSVC(C=C_value, loss='hinge')
    clf.fit(train,train_labels)
    ## Get predictions on training data
    train_preds = clf.predict(train)
    train_error = float(np.sum((train_preds > 0.0) != (train_labels > 0.0)))/len(train_labels)
        
    return train_error

In [26]:
cvals = [0.01,0.1,1.0,10.0,100.0,1000.0,10000.0]
for c in cvals:
    train_error = fit_classifier(c)
    print ("Error rate for C = %0.2f: train %0.3f" % (c, train_error))

Error rate for C = 0.01: train 0.020
Error rate for C = 0.10: train 0.000
Error rate for C = 1.00: train 0.000
Error rate for C = 10.00: train 0.000
Error rate for C = 100.00: train 0.000
Error rate for C = 1000.00: train 0.000
Error rate for C = 10000.00: train 0.000


Here, as we don't have test data the result is quickly without error.

Let's try to divide our labeled data into 80% training set and 20% testing set.

In [28]:
train = vect_df[0:84]
test = vect_df[84:105]
train_labels = data_cut['category'][0:84]
test_labels = data_cut['category'][84:105]

In [29]:
def fit_classifier1(C_value=1.0):
    clf = svm.LinearSVC(C=C_value, loss='hinge')
    clf.fit(train,train_labels)
    ## Get predictions on training data
    train_preds = clf.predict(train)
    train_error = float(np.sum((train_preds > 0.0) != (train_labels > 0.0)))/len(train_labels)
    ## Get predictions on test data
    test_preds = clf.predict(test)
    test_error = float(np.sum((test_preds > 0.0) != (test_labels > 0.0)))/len(test_labels)
    ##
    return train_error, test_error

In [36]:
cvals = [0.01,0.1,1.0,10.0,100.0,1000.0,10000.0]
for c in cvals:
    train_error, test_error = fit_classifier1(c)
    print ("Error rate for C = %0.2f: train %0.3f test %0.3f" % (c, train_error, test_error))

Error rate for C = 0.01: train 0.024 test 0.000
Error rate for C = 0.10: train 0.000 test 0.000
Error rate for C = 1.00: train 0.000 test 0.000
Error rate for C = 10.00: train 0.000 test 0.000
Error rate for C = 100.00: train 0.000 test 0.000
Error rate for C = 1000.00: train 0.000 test 0.000
Error rate for C = 10000.00: train 0.000 test 0.000


So we can choose C = 0.1

Now, let's make prediction for the rest of the dataset.

In [37]:
train = vect_df[0:105]
train_labels = data_cut['category'][0:105]

clf = svm.LinearSVC(C=0.1, loss='hinge')
clf.fit(train,train_labels)

    ## Get predictions on training data
data_to_predict = vect_df[105:1000]
preds = clf.predict(data_to_predict)
preds

array([1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2,
       1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 2,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 2, 2, 1, 1, 1, 1, 3, 1, 2, 2, 2, 2,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1,
       1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       2, 2, 1, 1, 0, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2,
       2, 1, 1, 1, 1, 3, 2, 3, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 3, 1, 2, 3, 2, 1,
       2, 3, 1, 3, 3, 1, 2, 1, 1, 2, 3, 1, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 3, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2,
       3, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2,
       1, 1, 1, 1, 3, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2,

There is a zeron so some data should be unlabelled, let's labelling them.

In [42]:
data_cut['clean_insight'][264]

'company produce unit particular computer component every month production cost company per component sell component end month minimum selling price per component guarantee yearly profit revenue sale minus production cost least xwhere sell cost one itemx profit one item number item produce sell per month number month yearsimplifying equation lead b'

In [46]:
for i in range (len(train_labels)):
    print(train_labels[i])

1
3
3
3
2
1
2
3
1
2
1
2
1
1
1
1
2
2
2
2
1
1
1
2
2
1
2
2
1
2
2
1
1
3
2
2
1
2
1
1
1
2
2
1
2
0
1
0
1
2
2
1
1
1
1
1
1
1
2
2
2
1
1
2
2
2
1
1
1
1
2
2
2
1
3
1
2
1
2
1
2
1
1
2
2
2
2
1
1
1
2
1
2
2
3
1
1
3
3
2
1
1
1
1
1


In [49]:
data_cut['category'][45]

0

In [50]:
data_cut['category'][47]

0

In [52]:
data_cut['counting and operation'][45]

0.0

In [53]:
data_cut['logical problem solving'][45]

0.0

In [55]:
data_cut['spatial representations'][45]

0.0

In [58]:
data_cut['question'][45]

'A shrewd opportunist bought 50 equally priced Kale Kids in November for t dollars total. In December, he sold each Kale Kid for a profit of 100 percent. In terms of t, how much was each Kale Kid sold for?'

In [69]:
data.loc[45,'logical problem solving'] = 1
data_cut.loc[45,'logical problem solving'] = 1
data_cut.loc[45,'category'] = 2

In [63]:
data_cut['question'][47]

'An agent gets a 5% commission on sales of cloths. In a certain day he got $500 as commission. The cloths sold through him on that day is worth?'

In [70]:
data.loc[47,'counting and operation'] = 1
data_cut.loc[47,'counting and operation'] = 1
data_cut.loc[47,'category'] = 1

In [71]:
train = vect_df[0:105]
train_labels = data_cut['category'][0:105]

clf = svm.LinearSVC(C=0.1, loss='hinge')
clf.fit(train,train_labels)

    ## Get predictions on training data
data_to_predict = vect_df[105:1000]
preds = clf.predict(data_to_predict)
preds

array([1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2,
       1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 2,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 2, 2, 1, 1, 1, 1, 3, 1, 2, 2, 2, 2,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1,
       1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2,
       2, 1, 1, 1, 1, 3, 2, 3, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 3, 1, 2, 3, 2, 1,
       2, 3, 1, 3, 3, 1, 2, 1, 1, 2, 3, 1, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 3, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2,
       3, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2,
       1, 1, 1, 1, 3, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2,